In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import time
import datetime
import random
import matplotlib.pylab as plt

#data collection
from bs4 import BeautifulSoup
import requests
import re

In [3]:
cities = ['Hayward', 'Fremont']

common_url = 'https://www.zillow.com/homes/for_sale/'

url_list = []
for city in cities:
    #url_list.append(common_url+city+"/ca") #for page 1
    url_list.append(common_url+city+"/ca/2_p/") #for page 2

In [4]:
req_headers = {
    'Aman-Solanki'
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Aman-Solanki'
}

s = requests.Session()
link=[] #list to store listing url
listingAgent=[] #list to store the listing company of the property
temp=[] #temperory list to append all the collected data

for url in url_list:
    r = s.get(url, headers=req_headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    for i in soup.findAll(class_="list-card-link list-card-link-top-margin list-card-img"):
        link.append(i.get('href')) #appends urls to link list
    for i in soup:
        listingCompany = soup.find_all('div', attrs={'class': 'list-card-truncate'})
        for i in listingCompany:
            company = i.text
            temp.append(company)
            temp = temp[:len(link)] #appends listing companies for only the accounted links

listingAgent.extend(temp) #adds all the temp elements to listingAgent list

In [5]:
print(len(link), 'links generated.')

80 links generated.


In [6]:
chunks = [link[x:x+8] for x in range(0, len(link), 8)] #creates 10 lists with 8 links in each list

In [7]:
len(chunks)

10

In [8]:
null_val = 'Not_Found' 
start_time = datetime.datetime.now()

print(start_time)

zillowDataList = [] #list to store dictionary generated for each property data.

s = requests.Session()

for i in range(len(chunks)):
    print('\n')
    print('Sleeping Now')
    print('------------------------------------------')
    print('\n')
    time.sleep(random.randint(26,30)) #sleeps after scraping 5 links
    for j in chunks[i]:
        zillowData = {}
        print('Sleeping Again....')
        time.sleep(random.randint(13,16)) #sleeps before scraping data from each link
        r = s.get(j, headers=req_headers)
        bs_html_data = BeautifulSoup(r.content, 'html.parser')

        print('Link: ', j)
        
        #price
        if bs_html_data.find('span', attrs={'class': 'ds-value'})!= None:
            price = re.sub("\D", "", bs_html_data.find('span', attrs={'class': 'ds-value'}).text) 
            zillowData['Price'] = price #adds price to dictionary 
        else:
            price = null_val
            zillowData['Price'] = price #adds null_val if price not found
        
        #address
        address_tags = bs_html_data.find_all('h1', attrs={'class': 'ds-address-container'})
            
        if address_tags != None:
            address_tags = bs_html_data.find_all('h1', attrs={'class': 'ds-address-container'})
            spans = address_tags[0].find_all('span')
                
        if spans != None:
            home_address = ' '.join([_.text for _ in spans])

            clean_home_address = re.sub(r'[^a-zA-Z0-9,_ ]', '', home_address)
            zillowData['Address'] = clean_home_address #full property address

            addr_line1 = clean_home_address.split(',')[0].strip() #address line 1

            home_city = clean_home_address.split(',')[-2].strip() #city
            zillowData['City'] = home_city #adds city to dictionary

            zipCode = int(clean_home_address.split(',')[-1].strip().split(' ')[-1].strip()) #zipcode
            zillowData['ZipCode'] = zipCode #adds zipcode to dictionary

            state = clean_home_address.split(',')[-1].strip().split(' ')[0].strip() #state
            zillowData['State'] = state #adds state to dictionary

        #zestimate
        zestimate = bs_html_data.find("p", attrs = {"class":"Text-aiai24-0 StyledParagraph-sc-18ze78a-0 eVIlQF qf5kuj-8 gfJJuM"})
        value=(re.sub("\D", "", zestimate.text) if zestimate != None else null_val)
        zillowData['Zestimate'] = value #adds zestimate to dictionary
        print('Zestimate:', value)

        
        #Interior details
        interior_details = bs_html_data.find_all('span', attrs={'class': 'Text-aiai24-0 gbgvjX'}) #get interior details

        if interior_details != None:
            bedrooms = (interior_details[0].text.split(' ')[1] if interior_details[0] != None else null_val)
            zillowData['Bedrooms'] = bedrooms #adds bedrooms to dictionary
        
        #adds the specified data to the dictionary, if data is not found, null_val is added instead
        for i in interior_details:
            text = i.text
            if text.startswith('Total interior livable area:'):
                area = text.split()[4] if text.split()[4] != None else null_val
                zillowData['Living-Area'] = area  
            if text.startswith('Bathrooms:'):
                tBathrooms = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['Total-Bathrooms'] = (int(tBathrooms))
                homeType = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['Home-Type'] = homeType 
            if text.startswith('Year built:'):
                yearBuilt = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['Year-Built'] = yearBuilt
            if text.startswith('Utilities for property:'):
                utilitiesProvider = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['Utilities'] = utilitiesProvider
            if text.startswith('Sunscore: Great solar potentialSun Number™:'):
                sunScore = text.split(':')[2] if text.split(':')[2] != None else null_val
                zillowData['Sunscore'] = sunScore
            if text.startswith('HOA fee: $'):
                listingHOA = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['HOA'] = listingHOA
            if text.startswith('Tax assessed value:'):
                taxValue = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['Tax-Value'] = taxValue
            if text.startswith('Annual tax amount:'):
                aTax = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['Annual-Tax'] = aTax
            if text.startswith('Garage spaces:'):
                gSpace = text.split(':')[1] if text.split(':')[1] != None else null_val
                zillowData['Garage-Spaces'] = gSpace  
        
        zillowDataList.append(zillowData) #appends the dictionary to the list



print('Total time:',datetime.datetime.now()-start_time, 'minutes') 

2020-10-28 14:49:04.087798


Sleeping Now
------------------------------------------


Sleeping Again....
Link:  https://www.zillow.com/homedetails/3322-D-St-Hayward-CA-94541/24967635_zpid/
Zestimate: Not_Found
Sleeping Again....
Link:  https://www.zillow.com/community/promontory-at-stonebrae/2078676723_zpid/
Zestimate: Not_Found


IndexError: list index out of range

In [ ]:
zillow = pd.DataFrame(zillowDataList, index= range(len(zillowDataList))) #creates a dataFrame the list of dictionaries
zillow['Link'] = link #adds link column to the data frame
zillow['Listing-Company'] = listingAgent #adds listingCompany column to the data frame

In [ ]:
zillow.head(2)

In [ ]:
zillow = zillow[['Address', 'City', 'ZipCode', 'State', 'Bedrooms','Price',
                  'Listing-Company', 'HOA', 'Zestimate', 'Link',
                  'Living-Area','Total-Bathrooms', 'Full-Bathrooms',
                  'Half-Bathrooms', 'Garage-Spaces','Stories', 'Fireplace',
                  'Lot-Size', 'Home-Type', 'New-Contruction','Year-Built',
                  'Utilities', 'Sunscore', 'Annual-Tax', 'Tax-Value']]